In [1]:
import pandas as pd
import os
from PIL import Image
from keras.models import Model
from keras.applications import VGG16, ResNet50
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import seaborn as sns
from sklearn.utils import shuffle, resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
import tensorflow.keras as keras
from keras import layers
import matplotlib.pyplot as plt
from tabulate import tabulate
import joblib
from sklearn.svm import SVC
from skimage import color, io, transform
import pickle
class_label = ['Non-Default(0)','Default(1)'] # env var

In [2]:
data = pd.read_csv('D:\Barath Suresh Docs\PROGRAMMING\MACHINE LEARNING\credit_card_fraud_detection\creditcard.csv')

In [3]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data.drop('Class', axis=1), data['Class'], test_size=0.2, random_state=42)

In [8]:
# Load the VGG16 model
vgg_model = VGG16(include_top=False, input_shape=(224,224,3))

In [12]:
# Generate image features using the VGG16 model
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(dataframe=X_train, y_col='Class',
                                                    batch_size=32, shuffle=True, class_mode='raw')
train_features = vgg_model.predict(train_generator)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(dataframe=X_test, y_col='Class',
                                                  batch_size=32, shuffle=False, class_mode='raw')
test_features = vgg_model.predict(test_generator)

KeyError: 'filename'

In [ ]:
# Train an SVM classifier on the generated image features
svm_classifier = SVC()
svm_classifier.fit(train_features, y_train)

In [ ]:
# Evaluate the SVM classifier on the testing set
svm_predictions = svm_classifier.predict(test_features)
svm_accuracy = accuracy_score(y_test, svm_predictions)
print('SVM accuracy: {:.2f}%'.format(svm_accuracy * 100))

In [ ]:
# Create a new model using the VGG16 base and a new output layer
new_model = Sequential()
new_model.add(vgg_model)
new_model.add(Flatten())
new_model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile the new model
new_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the new model on the credit card fraud dataset
new_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# Evaluate the new model on the testing set
new_model_accuracy = new_model.evaluate(X_test, y_test)[1]
print('New model accuracy: {:.2f}%'.format(new_model_accuracy * 100))